In [1]:
import sys
sys.path.append('/Users/madsbertelsen/PaNOSC/McStasScript') # Path to McStasScript pythoon file

from mcstasscript.interface import instr, plotter, functions

# Creating the instance of the class, insert path to mcrun and to mcstas root directory
detector = instr.McStas_instr("LOKI_detector",
                              mcrun_path = "/Applications/McStas-2.5.app/Contents/Resources/mcstas/2.5/bin",
                              mcstas_path = "/Applications/McStas-2.5.app/Contents/Resources/mcstas/2.5")

In [2]:
detector.add_parameter("wavelength")
detector.add_parameter("height",value=1.0,comment="Height in [m]")
detector.add_parameter("string","reflection_filename")
detector.add_parameter("string","data_filename",value="\"data.dat\"")

In [3]:
detector.add_declare_var("double","energy")
detector.add_declare_var("int","flag")
detector.add_declare_var("double","tube_radius",value=0.013)
detector.add_declare_var("double","displacements",array=7)
detector.add_declare_var("double","t_array",array=4,value=[0.65E-6,0.65E-6,1E-6,1E-6])

In [4]:
detector.append_initialize("energy=pow(2*PI/wavelength*K2V,2)*VS2E;")

In [5]:
detector.add_component("Origin","Arm")
src = detector.add_component("source","Source_simple",RELATIVE="Origin")
detector.add_component("beam_extraction","Guide_gravity",AT=[0,0,2],RELATIVE="source")

In [6]:
detector.add_component("pre_guide_slit","Slit",before="beam_extraction",
                       AT=[0,0,1],RELATIVE="source",comment="Slit before the guide")

In [7]:
detector.print_components()

Origin            Arm             AT  [0, 0, 0]     ABSOLUTE          ROTATED  [0, 0, 0]     ABSOLUTE
source            Source_simple   AT  [0, 0, 0]     RELATIVE Origin   ROTATED  [0, 0, 0]     RELATIVE Origin
pre_guide_slit    Slit            AT  [0, 0, 1]     RELATIVE source   ROTATED  [0, 0, 0]     RELATIVE source
beam_extraction   Guide_gravity   AT  [0, 0, 2]     RELATIVE source   ROTATED  [0, 0, 0]     RELATIVE source


In [8]:
detector.set_component_parameter("source",{"xwidth" : 0.12, "E0" : "energy"})
detector.set_component_parameter("source",{"yheight" : 0.12})

In [9]:
detector.print_component("source")

COMPONENT source = Source_simple
  yheight = 0.12 [m]
  xwidth = 0.12 [m]
  E0 = energy [meV]
AT [0, 0, 0] RELATIVE Origin
ROTATED [0, 0, 0] RELATIVE Origin


In [10]:
detector.set_component_AT("source",[0.01,0,0])

In [11]:
detector.set_component_ROTATED("beam_extraction",[0,2.0,0],RELATIVE="Origin")

In [12]:
detector.set_component_RELATIVE("beam_extraction","pre_guide_slit")

In [20]:
detector.print_components()

Origin            Arm             AT  [0, 0, 0]        ABSOLUTE                  ROTATED  [0, 0, 0]       ABSOLUTE
source            Source_simple   AT  [0.01, 0, 0]     RELATIVE Origin           ROTATED  [0, 0, 0]       RELATIVE Origin
pre_guide_slit    Slit            AT  [0, 0, 1]        RELATIVE source           ROTATED  [0, 0, 0]       RELATIVE source
beam_extraction   Guide_gravity   AT  [0, 0, 2]        RELATIVE pre_guide_slit   ROTATED  [0, 2.0, 0]     RELATIVE pre_guide_slit


In [13]:
detector.set_component_WHEN("beam_extraction","vx > 0")
detector.print_component("beam_extraction")

COMPONENT beam_extraction = Guide_gravity
  w1 : Required parameter not yet specified
  h1 : Required parameter not yet specified
  l : Required parameter not yet specified
WHEN (vx > 0)
AT [0, 0, 2] RELATIVE pre_guide_slit
ROTATED [0, 2.0, 0] RELATIVE pre_guide_slit


In [14]:
detector.append_component_EXTEND("beam_extraction","n_scattering = SCATTERED - 2")
detector.print_component("beam_extraction")

COMPONENT beam_extraction = Guide_gravity
  w1 : Required parameter not yet specified
  h1 : Required parameter not yet specified
  l : Required parameter not yet specified
WHEN (vx > 0)
AT [0, 0, 2] RELATIVE pre_guide_slit
ROTATED [0, 2.0, 0] RELATIVE pre_guide_slit
EXTEND %{
n_scattering = SCATTERED - 2
%}


In [15]:
detector.set_component_GROUP("beam_extraction","guides")
detector.print_component("beam_extraction")

COMPONENT beam_extraction = Guide_gravity
  w1 : Required parameter not yet specified
  h1 : Required parameter not yet specified
  l : Required parameter not yet specified
WHEN (vx > 0)
AT [0, 0, 2] RELATIVE pre_guide_slit
ROTATED [0, 2.0, 0] RELATIVE pre_guide_slit
GROUP guides
EXTEND %{
n_scattering = SCATTERED - 2
%}


In [16]:
detector.set_component_JUMP("beam_extraction","myself iterate 3")
detector.print_component("beam_extraction")

COMPONENT beam_extraction = Guide_gravity
  w1 : Required parameter not yet specified
  h1 : Required parameter not yet specified
  l : Required parameter not yet specified
WHEN (vx > 0)
AT [0, 0, 2] RELATIVE pre_guide_slit
ROTATED [0, 2.0, 0] RELATIVE pre_guide_slit
GROUP guides
EXTEND %{
n_scattering = SCATTERED - 2
%}
JUMP myself iterate 3


In [17]:
detector.set_component_comment("beam_extraction","Simulating severe misalignment")
detector.print_component("beam_extraction")

// Simulating severe misalignment
COMPONENT beam_extraction = Guide_gravity
  w1 : Required parameter not yet specified
  h1 : Required parameter not yet specified
  l : Required parameter not yet specified
WHEN (vx > 0)
AT [0, 0, 2] RELATIVE pre_guide_slit
ROTATED [0, 2.0, 0] RELATIVE pre_guide_slit
GROUP guides
EXTEND %{
n_scattering = SCATTERED - 2
%}
JUMP myself iterate 3


In [18]:
detector.show_components()

Here are the availalbe component categories:
 sources
 optics
 samples
 monitors
 misc
 contrib
 union
 obsolete
Call show_components(category_name) to display


In [19]:
detector.show_components("samples")

Here are all components in the samples category.
 Incoherent      Phonon_simple   Res_sample      Single_crystal
 Isotropic_Sqw   Powder1         Sans_spheres    TOFRes_sample
 Magnon_bcc      PowderN         SasView_model   Tunneling_sample


In [20]:
detector.component_help("Phonon_simple")

 ___ Help Phonon_simple _________________________________________________
|optional parameter|required parameter|default value|user specified value|
radius [m] // Outer radius of sample in (x,z) plane
yheight [m] // Height of sample in y direction
sigma_abs [barns] // Absorption cross section at 2200 m/s per atom
sigma_inc [barns] // Incoherent scattering cross section per atom
a [AA] // fcc Lattice constant
b [fm] // Scattering length
M [a.u.] // Atomic mass
c [meV/AA^(-1)] // Velocity of sound
DW [1] // Debye-Waller factor
T [K] // Temperature
target_x = 0 [m] // position of target to focus at . Transverse coordinate
target_y = 0 [m] // position of target to focus at. Vertical coordinate
target_z = 0 [m] // position of target to focus at. Straight ahead.
target_index = 0 [1] // relative index of component to focus at, e.g. next is +1
focus_r = 0 [m] // Radius of sphere containing target.
focus_xw = 0 [m] // horiz. dimension of a rectangular area
focus_yh = 0 [m] // vert.  dimension o